## SAFE RESEARCH & REPORTING ASSISTANT
In this notebook we'll bring together our learnings on generative AI agents and tooling, and code an AI agent which incorporates a search engine for doing research using up to date information and a guardrails for protecting against banned topics. We'll use a generative AI model to write a report. We'll use the smolagent framework to construct the AI agent.

To start, let's suppress any unecessary warnings.

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

Next, let's import the smolagents library.

In [ ]:
%pip install -q smolagents

We can now import the libraries we'll be using for our tools and agents.

In [3]:
from smolagents import CodeAgent, HfApiModel, DuckDuckGoSearchTool, tool
import os, requests
session = requests.Session()  # Allows persistent connection

Now we can start to create our tools.  We will use DuckDuckGo to search the web directly from the library, so we don't need to code it.

The next tool we'll define is our response guardrail.  We'll use the decorator and comments to start, and then define the _scanner_ handle with the topics we're interested in banning and the threshold for triggering the guardrail.  We then run the scanner and get back the results of the scanning.

In [8]:
@tool
def out_guard(article: str)->str:
  """This is a guardrail to check the article for any banned topics. The banned topics are violence, religion or politics.
     It will return True if the article contains banned topics, and False if it does not. If it contains banned topics
     then the article should not be used.
     Args:
       article: The search results that could potentially contain violence, politics, or religion
  """
  check = session.post(
    "https://api.lakera.ai/v2/guard",
    json={"messages": [{"role": "user", "content": article}]},
    headers={"Authorization": f'Bearer {os.getenv("LAKERA_API_KEY")}'},
  )
  print("Guard check is ", check.json()['flagged'])
  return check.json()['flagged']

We can now construct our agent.  We're specifying an agent which generates code and runs it to integrate the tooling. We're specifying the tools to use and directing the report wiriting to be done using the default Huggingface Qwen AI model.

In [9]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool(),out_guard],model=HfApiModel(""))

We can now run the agent with a prompt.

In [ ]:
prompt="Search the web to get information for a research report on the latest developments in Quantum Computing. When you get your results make sure they do not include banned topics. Write the report and use markdown to format it."
result = agent.run(prompt)

If we check the run log of the agent, we can see where the tools were invoked, and we get the final result that we requested.  We have saved the output and can reprint it.

In [ ]:
print(result)